# <center> Model Gini Calculation </center>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# Function

## calculate_gini_for_threedigitscore

In [2]:
# def calculate_gini_for_threedigitscore(scores, labels):
#     """
#     Calculate Gini coefficient for three-digit scores and binary labels
    
#     Parameters:
#     scores: array-like, three-digit scores (higher is better)
#     labels: array-like, binary values (0 or 1, where 1 indicates default)
    
#     Returns:
#     float: Gini coefficient
#     """
#     # Combine scores and labels into a DataFrame
#     df = pd.DataFrame({'score': scores, 'label': labels})
    
#     # Sort by score in descending order (assuming higher score is better)
#     df = df.sort_values('score', ascending=False)
    
#     # Calculate cumulative values
#     total_pos = df['label'].sum()
#     total_neg = len(df) - total_pos
    
#     if total_pos == 0 or total_neg == 0:
#         return 0
    
#     # Calculate cumulative proportions
#     cum_pos = df['label'].cumsum()
#     cum_neg = np.arange(1, len(df) + 1) - cum_pos
    
#     # Convert to proportions
#     cum_pos_prop = cum_pos / total_pos
#     cum_neg_prop = cum_neg / total_neg
    
#     # Calculate Gini
#     gini = 1 - np.trapz(cum_pos_prop, cum_neg_prop)
    
#     return gini


## Modified one

def calculate_gini_for_threedigitscore(scores, labels):
    """
    Calculate Gini coefficient for three-digit scores and binary labels
    
    Parameters:
    scores: array-like, three-digit scores (higher is better)
    labels: array-like, binary values (0 or 1, where 1 indicates default)
    
    Returns:
    float: Gini coefficient
    """
    # Combine scores and labels into a DataFrame
    df = pd.DataFrame({'score': scores, 'label': labels})
    
    # Sort by score in descending order (assuming higher score means lower risk)
    # For default prediction, we want to sort scores in ascending order 
    # since higher default probability should correspond to higher risk
    df = df.sort_values('score', ascending=True)  # Changed to ascending=True
    
    # Calculate cumulative values
    total_pos = df['label'].sum()
    total_neg = len(df) - total_pos
    
    if total_pos == 0 or total_neg == 0:
        return 0
    
    # Calculate cumulative proportions
    cum_pos = df['label'].cumsum()
    cum_neg = np.arange(1, len(df) + 1) - cum_pos
    
    # Convert to proportions
    cum_pos_prop = cum_pos / total_pos
    cum_neg_prop = cum_neg / total_neg
    
    # Calculate area under curve
    auc = np.trapz(cum_pos_prop, cum_neg_prop)
    
    # Calculate Gini
    gini = 2 * auc - 1
    
    return gini

## calculate_gini

In [3]:
def calculate_gini(pd_scores, bad_indicators):
    """
    Calculate Gini coefficient from scores and binary indicators
    
    Parameters:
    pd_scores: array-like of scores/probabilities
    bad_indicators: array-like of binary outcomes (0/1)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays and ensure they're numeric
    pd_scores = np.array(pd_scores, dtype=float)
    bad_indicators = np.array(bad_indicators, dtype=int)
    
    # Check for valid input data
    if len(pd_scores) == 0 or len(bad_indicators) == 0:
        return np.nan
    
    # Check if we have both good and bad cases (needed for ROC AUC)
    if len(np.unique(bad_indicators)) < 2:
        return np.nan
    
    # Calculate AUC using sklearn
    try:
        auc = roc_auc_score(bad_indicators, pd_scores)
        # Calculate Gini from AUC
        gini = 2 * auc - 1
        return gini
    except ValueError:
        return np.nan

## calculate_hybrid_gini

In [4]:
# def calculate_hybrid_gini(scores, labels):
#     """
#     Calculate Gini coefficient handling both PD values and three-digit scores
    
#     Parameters:
#     scores: array-like, contains either PD values (0-1) or three-digit scores
#     labels: array-like, binary values (0 or 1, where 1 indicates default)
    
#     Returns:
#     float: Gini coefficient
#     """
#     # Convert inputs to numpy arrays
#     scores = np.array(scores, dtype=float)
#     labels = np.array(labels, dtype=int)
    
#     # Basic validation
#     if len(scores) == 0 or len(labels) == 0:
#         return np.nan
    
#     if len(np.unique(labels)) < 2:
#         return np.nan
        
#     # Determine if scores are PD values or three-digit scores
#     # PD values are between 0 and 1
#     is_pd = np.all((scores >= 0) & (scores <= 1))
    
#     if is_pd:
#         try:
#             auc = roc_auc_score(labels, scores)
#             gini = 2 * auc - 1
#             return gini
#         except ValueError:
#             return np.nan
#     else:
#         # Handle as three-digit score
#         df = pd.DataFrame({'score': scores, 'label': labels})
#         df = df.sort_values('score', ascending=False)
        
#         total_pos = df['label'].sum()
#         total_neg = len(df) - total_pos
        
#         if total_pos == 0 or total_neg == 0:
#             return np.nan
        
#         cum_pos = df['label'].cumsum()
#         cum_neg = np.arange(1, len(df) + 1) - cum_pos
        
#         cum_pos_prop = cum_pos / total_pos
#         cum_neg_prop = cum_neg / total_neg
        
#         gini = 1 - np.trapz(cum_pos_prop, cum_neg_prop)
#         return gini

## Modified one

def calculate_hybrid_gini(scores, labels):
    """
    Calculate Gini coefficient handling both PD values and three-digit scores
    
    Parameters:
    scores: array-like, contains either PD values (0-1) or three-digit scores
    labels: array-like, binary values (0 or 1, where 1 indicates default)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays
    scores = np.array(scores, dtype=float)
    labels = np.array(labels, dtype=int)
    
    # Basic validation
    if len(scores) == 0 or len(labels) == 0:
        return np.nan
    
    if len(np.unique(labels)) < 2:
        return np.nan
        
    # Determine if scores are PD values or three-digit scores
    # PD values are between 0 and 1
    is_pd = np.all((scores >= 0) & (scores <= 1))
    
    if is_pd:
        try:
            auc = roc_auc_score(labels, scores)
            gini = 2 * auc - 1
            return gini
        except ValueError:
            return np.nan
    else:
        # Handle as three-digit score
        df = pd.DataFrame({'score': scores, 'label': labels})
        # Sort by score in ascending order since higher score means higher risk
        df = df.sort_values('score', ascending=True)
        
        total_pos = df['label'].sum()
        total_neg = len(df) - total_pos
        
        if total_pos == 0 or total_neg == 0:
            return np.nan
        
        cum_pos = df['label'].cumsum()
        cum_neg = np.arange(1, len(df) + 1) - cum_pos
        
        cum_pos_prop = cum_pos / total_pos
        cum_neg_prop = cum_neg / total_neg
        
        # Calculate area under curve
        auc = np.trapz(cum_pos_prop, cum_neg_prop)
        
        # Calculate Gini using the same formula as PD values
        gini = 2 * auc - 1
        return gini

## calculate_periodic_gini_threedigit

In [5]:
# Main processing code
def calculate_periodic_gini_threedigit(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_gini_for_threedigitscore(x[score_column], x[label_column])
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_gini_for_threedigitscore(x[score_column], x[label_column])
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

## calculate_periodic_gini

In [6]:
def calculate_periodic_gini(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
        
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_gini(x[score_column], x[label_column])
        if len(x) >= 10 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_gini(x[score_column], x[label_column])
        if len(x) >= 20 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

## calculate_periodic_hybrid_gini

In [7]:
def calculate_periodic_hybrid_gini(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients for mixed score types
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
        
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_hybrid_gini(x[score_column], x[label_column])
        if len(x) >= 10 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + pd.Timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_hybrid_gini(x[score_column], x[label_column])
        if len(x) >= 20 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

# App Score FPD10

In [8]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_apps_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    s_apps_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from appscore;"""

dfappscorefpd10 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 45e969da-1f74-4171-8dde-e4d49e37fe50 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [9]:
dfappscorefpd10.sample(5)

,disbursementdate,digitalLoanAccountId,s_apps_score,ln_fpd10_flag,ln_mature_fpd10_flag
138384,2023-10-22 15:25:04,617ef401-4412-40ae-a01b-26099bb593c1,0.528587,0,1
46091,2024-04-17 10:33:31,24cc0606-ef24-48ba-af06-484515fe5af2,0.615571,0,1
155656,2024-07-25 11:31:38,c589fad1-c912-4309-a8ef-cc473d657764,0.513535,0,1
112590,2024-12-26 15:01:29,0be88b43-b9d4-4d02-b32f-579e3d39501f,0.661346,0,1
133524,2023-09-28 18:10:26,29127657-92df-4bfc-93cb-170fb709b232,0.405437,0,1


In [10]:
gini_results = calculate_periodic_gini(dfappscorefpd10, 's_apps_score', 'ln_fpd10_flag', 'FPD10')

In [11]:
gini_results.head()

,start_date,end_date,s_apps_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.548230,Week,s_apps_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.384648,Month,s_apps_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.500000,Week,s_apps_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.555195,Week,s_apps_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,0.004831,Week,s_apps_score,1.1.0,FPD10


In [12]:
appscoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFPD10.shape}")
appscoreFPD10.columns.values

The shape of dataframe after copy is:	(106, 7)


array(['start_date', 'end_date', 's_apps_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [13]:
gini_results.head()

,start_date,end_date,s_apps_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.548230,Week,s_apps_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.384648,Month,s_apps_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.500000,Week,s_apps_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.555195,Week,s_apps_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,0.004831,Week,s_apps_score,1.1.0,FPD10


# App Score FPD30

In [14]:
sq = """
with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_apps_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    s_apps_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from appscore;
"""

dfappscorefpd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 7af57b01-7b68-4704-b9ef-7c458e9f6570 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [15]:
dfappscorefpd30.head()

,disbursementdate,digitalLoanAccountId,s_apps_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-06-01 10:42:34,f91cba31-7630-477a-8cdd-d010b63c4900,0.370618,0,1
1,2023-06-01 11:02:18,7e7e7678-b36d-4b26-ad01-272565b54780,0.483115,0,1
2,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.411007,0,1
3,2023-06-01 14:43:46,4e78ebf3-256e-426c-8840-a7f9e1403a97,0.501267,0,1
4,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.550104,0,1


In [16]:
gini_results = calculate_periodic_gini(dfappscorefpd30, 's_apps_score', 'ln_fpd30_flag', 'FPD30')
# gini_results['bad_rate'] = 'FPD30'
appscoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFPD30.shape}")
appscoreFPD30.columns.values

The shape of dataframe after copy is:	(102, 7)


array(['start_date', 'end_date', 's_apps_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [17]:
gini_results.sample(5)

,start_date,end_date,s_apps_score_FPD30_gini,period,Model_Name,version,bad_rate
98,2024-12-02,2024-12-08,0.329144,Week,s_apps_score,1.1.0,FPD30
37,2023-12-25,2023-12-31,0.417435,Week,s_apps_score,1.1.0,FPD30
49,2024-03-01,2024-03-31,0.413533,Month,s_apps_score,1.1.0,FPD30
20,2023-09-18,2023-09-24,0.169415,Week,s_apps_score,1.1.0,FPD30
27,2023-10-30,2023-11-05,0.391366,Week,s_apps_score,1.1.0,FPD30


# App Score FSPD30

In [18]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_apps_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    s_apps_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from appscore;
"""

dfappscorefspd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 656ea9d7-dae6-4105-a1f8-5ad5ca963b4a successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [19]:
gini_results = calculate_periodic_gini(dfappscorefspd30, 's_apps_score', 'ln_fspd30_flag', 'FSPD30')
# gini_results['bad_rate'] = 'FSPD30'
appscoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFSPD30.shape}")
appscoreFSPD30.columns.values

The shape of dataframe after copy is:	(97, 7)


array(['start_date', 'end_date', 's_apps_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [20]:
gini_results.tail()

,start_date,end_date,s_apps_score_FSPD30_gini,period,Model_Name,version,bad_rate
92,2024-11-01,2024-11-30,0.298595,Month,s_apps_score,1.1.0,FSPD30
93,2024-11-04,2024-11-10,0.295698,Week,s_apps_score,1.1.0,FSPD30
94,2024-11-11,2024-11-17,0.306730,Week,s_apps_score,1.1.0,FSPD30
95,2024-11-18,2024-11-24,0.277736,Week,s_apps_score,1.1.0,FSPD30
96,2024-11-25,2024-12-01,0.245972,Week,s_apps_score,1.1.0,FSPD30


# App Score FSTPD30

In [21]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_apps_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    s_apps_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from appscore;
"""

dfappscorefstpd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID fa27dab1-8554-4eb5-9027-8744f67f02be successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [22]:
gini_results = calculate_periodic_gini(dfappscorefstpd30, 's_apps_score', 'ln_fstpd30_flag', 'FSTPD30')
# gini_results['bad_rate'] = 'FSTPD30'
appscoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFSTPD30.shape}")
appscoreFSTPD30.columns.values

The shape of dataframe after copy is:	(91, 7)


array(['start_date', 'end_date', 's_apps_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [23]:
appscoreFSTPD30.head()

,start_date,end_date,s_apps_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.465098,Week,s_apps_score,1.1.0,FSTPD30
1,2023-06-01,2023-06-30,0.337420,Month,s_apps_score,1.1.0,FSTPD30
2,2023-06-05,2023-06-11,0.413300,Week,s_apps_score,1.1.0,FSTPD30
3,2023-06-12,2023-06-18,0.395676,Week,s_apps_score,1.1.0,FSTPD30
4,2023-06-19,2023-06-25,0.314497,Week,s_apps_score,1.1.0,FSTPD30


# Combining App Score

In [24]:
import functools

dataframes = [appscoreFPD10, appscoreFPD30, appscoreFSPD30, appscoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 's_apps_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 's_apps_score_FPD30_gini',
       's_apps_score_FSPD30_gini', 's_apps_score_FSTPD30_gini'],
      dtype=object)

In [25]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','s_apps_score_FSTPD30_gini','s_apps_score_FSPD30_gini',
       's_apps_score_FPD30_gini', 's_apps_score_FPD10_gini']].copy()
final_df.dtypes

start_date                   datetime64[ns]
end_date                     datetime64[ns]
period                               object
Model_Name                           object
version                              object
bad_rate                             object
s_apps_score_FSTPD30_gini           float64
s_apps_score_FSPD30_gini            float64
s_apps_score_FPD30_gini             float64
s_apps_score_FPD10_gini             float64
dtype: object

## Creating app score table 

In [26]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_apps_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=256bef3a-35e6-442b-ab53-064742035d6f>

In [27]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('s_apps_score_FSTPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_apps_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_apps_score_FPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_apps_score_fpd10_gini', 'FLOAT')
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_apps_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=36ea271f-d6e6-4bd5-8d0f-7a30262d06b3>

# sb_demo_score

## FPD10

In [28]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_demo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    sb_demo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID cc006fcb-a356-40cf-aa35-9fce466d77b0 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [29]:
df_sb_demo_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,sb_demo_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2023-06-01 10:42:34,f91cba31-7630-477a-8cdd-d010b63c4900,0.095033,0,1
1,2023-06-01 11:02:18,7e7e7678-b36d-4b26-ad01-272565b54780,0.136369,0,1
2,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.113972,0,1
3,2023-06-01 14:43:46,4e78ebf3-256e-426c-8840-a7f9e1403a97,0.154295,0,1
4,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.130198,0,1


In [30]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefpd10, 'sb_demo_score', 'ln_fpd10_flag', 'FPD10')
sb_demo_scoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFPD10.shape}")
sb_demo_scoreFPD10.columns.values

The shape of dataframe after copy is:	(106, 7)


array(['start_date', 'end_date', 'sb_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [31]:
sb_demo_scoreFPD10.tail()

,start_date,end_date,sb_demo_score_FPD10_gini,period,Model_Name,version,bad_rate
101,2024-12-23,2024-12-29,0.236387,Week,sb_demo_score,1.1.0,FPD10
102,2024-12-30,2025-01-05,0.254618,Week,sb_demo_score,1.1.0,FPD10
103,2025-01-01,2025-01-31,0.242613,Month,sb_demo_score,1.1.0,FPD10
104,2025-01-06,2025-01-12,0.172630,Week,sb_demo_score,1.1.0,FPD10
105,2025-01-13,2025-01-19,0.444206,Week,sb_demo_score,1.1.0,FPD10


## FPD30

In [32]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_demo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    sb_demo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 26b6cd24-e8e9-43fc-9576-882ca0c1ddc0 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [33]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefpd30, 'sb_demo_score', 'ln_fpd30_flag', 'FPD30')
sb_demo_scoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFPD30.shape}")
sb_demo_scoreFPD30.columns.values

The shape of dataframe after copy is:	(102, 7)


array(['start_date', 'end_date', 'sb_demo_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [34]:
sb_demo_scoreFPD30.head() 

,start_date,end_date,sb_demo_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.330909,Week,sb_demo_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,0.196615,Month,sb_demo_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.150370,Week,sb_demo_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,0.374306,Week,sb_demo_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,-0.064257,Week,sb_demo_score,1.1.0,FPD30


## FSPD30

In [35]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_demo_score,
    ln_fspd30_flag,   -- fspd30
	ln_mature_fspd30_flag,	--- fspd30 observation
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    sb_demo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 1c6b508a-7824-4779-91fd-418a48d3b28b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [36]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefspd30, 'sb_demo_score', 'ln_fspd30_flag', 'FSPD30')
sb_demo_scoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFSPD30.shape}")
sb_demo_scoreFSPD30.columns.values

The shape of dataframe after copy is:	(97, 7)


array(['start_date', 'end_date', 'sb_demo_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [37]:
sb_demo_scoreFSPD30.head()

,start_date,end_date,sb_demo_score_FSPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.126685,Week,sb_demo_score,1.1.0,FSPD30
1,2023-06-01,2023-06-30,0.184672,Month,sb_demo_score,1.1.0,FSPD30
2,2023-06-05,2023-06-11,0.239192,Week,sb_demo_score,1.1.0,FSPD30
3,2023-06-12,2023-06-18,0.313433,Week,sb_demo_score,1.1.0,FSPD30
4,2023-06-19,2023-06-25,-0.008772,Week,sb_demo_score,1.1.0,FSPD30


## FSTPD30

In [38]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_demo_score,
    ln_fstpd30_flag,   -- fstpd30
	ln_mature_fstpd30_flag,	--- fstpd30 observation
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    sb_demo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID dcf03d7f-33b1-4225-ae2a-753cafd80ead successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [39]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefstpd30, 'sb_demo_score', 'ln_fstpd30_flag', 'FSTPD30')
sb_demo_scoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFSTPD30.shape}")
sb_demo_scoreFSTPD30.columns.values

The shape of dataframe after copy is:	(91, 7)


array(['start_date', 'end_date', 'sb_demo_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [40]:
sb_demo_scoreFSTPD30.head()

,start_date,end_date,sb_demo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.241830,Week,sb_demo_score,1.1.0,FSTPD30
1,2023-06-01,2023-06-30,0.162500,Month,sb_demo_score,1.1.0,FSTPD30
2,2023-06-05,2023-06-11,0.272886,Week,sb_demo_score,1.1.0,FSTPD30
3,2023-06-12,2023-06-18,0.135395,Week,sb_demo_score,1.1.0,FSTPD30
4,2023-06-19,2023-06-25,0.074133,Week,sb_demo_score,1.1.0,FSTPD30


# Combining sb demo score

In [41]:
import functools

dataframes = [sb_demo_scoreFPD10, sb_demo_scoreFPD30, sb_demo_scoreFSPD30, sb_demo_scoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sb_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'sb_demo_score_FPD30_gini',
       'sb_demo_score_FSPD30_gini', 'sb_demo_score_FSTPD30_gini'],
      dtype=object)

In [42]:
final_df = final_df[['start_date', 'end_date', 'period',
       'Model_Name', 'version', 'bad_rate','sb_demo_score_FPD10_gini','sb_demo_score_FPD30_gini',
       'sb_demo_score_FSPD30_gini', 'sb_demo_score_FSTPD30_gini']].copy()

## creating sb demo score table 

In [43]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_demo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a062ff72-5a6b-4627-ba19-1e03790fd333>

In [44]:


import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('sb_demo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('sb_demo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('sb_demo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('sb_demo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_demo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=76d644d5-6379-4340-b41e-a3de3f454067>

# s_cic_score

## FPD10

In [45]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_cic_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    s_cic_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 1d5ad047-3bfb-4e72-baee-18e9b2c93f7e successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [46]:
df_s_cic_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,s_cic_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2023-06-02 13:49:38,69f07f7a-77ce-4f3d-87da-cf27b346ac52,0.086317,0,1
1,2023-06-02 14:43:59,b2770138-2905-48d7-a9bf-7af35014a465,0.177167,0,1
2,2023-06-02 15:41:58,6a98d1eb-50d9-4f70-83db-5297bc452475,0.093217,0,1
3,2023-06-03 11:35:43,5e3ca494-7918-406b-b437-a76164ff4ba6,0.067339,0,1
4,2023-06-03 11:44:25,9088cf01-f299-473c-9b29-23c49d289bc8,0.137690,1,1


In [47]:
gini_results = calculate_periodic_gini(df_s_cic_scorefpd10, 's_cic_score', 'ln_fpd10_flag', 'FPD10')
s_cic_scoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFPD10.shape}")
s_cic_scoreFPD10.columns.values

The shape of dataframe after copy is:	(105, 7)


array(['start_date', 'end_date', 's_cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [48]:
s_cic_scoreFPD10.tail()

,start_date,end_date,s_cic_score_FPD10_gini,period,Model_Name,version,bad_rate
100,2024-12-23,2024-12-29,0.156457,Week,s_cic_score,1.1.0,FPD10
101,2024-12-30,2025-01-05,0.225236,Week,s_cic_score,1.1.0,FPD10
102,2025-01-01,2025-01-31,0.189401,Month,s_cic_score,1.1.0,FPD10
103,2025-01-06,2025-01-12,0.166526,Week,s_cic_score,1.1.0,FPD10
104,2025-01-13,2025-01-19,0.075341,Week,s_cic_score,1.1.0,FPD10


## FPD30

In [49]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_cic_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    s_cic_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 5f9c493a-06ce-4cc0-8c92-a78ec8953615 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [50]:
df_s_cic_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,s_cic_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-06-01 11:02:18,7e7e7678-b36d-4b26-ad01-272565b54780,0.086860,0,1
1,2023-06-02 14:41:10,832f93ae-46bb-4fdf-94ff-66db94b716c9,0.120927,1,1
2,2023-06-02 15:33:58,531e98fe-749e-471e-9960-6dd80d55e86f,0.141870,1,1
3,2023-06-02 16:25:26,5dae7da0-f426-4402-b552-f84ab33d38bf,0.083796,1,1
4,2023-06-02 17:11:42,1d7abbe4-a62a-4f4b-91b2-bb360b9e61c2,0.142315,0,1


In [51]:
gini_results = calculate_periodic_gini(df_s_cic_scorefpd30, 's_cic_score', 'ln_fpd30_flag', 'FPD30')
s_cic_scoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFPD30.shape}")
s_cic_scoreFPD30.columns.values

The shape of dataframe after copy is:	(101, 7)


array(['start_date', 'end_date', 's_cic_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [52]:
s_cic_scoreFPD30.head()

,start_date,end_date,s_cic_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.523077,Week,s_cic_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,0.248378,Month,s_cic_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.437229,Week,s_cic_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,0.514851,Week,s_cic_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,0.035484,Week,s_cic_score,1.1.0,FPD30


In [53]:
s_cic_scoreFPD30.describe()

,start_date,end_date,s_cic_score_FPD30_gini
count,101,101,101.000000
mean,2024-03-11 12:21:23.168316928,2024-03-21 22:34:27.326732800,0.327125
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.035484
25%,2023-10-23 00:00:00,2023-10-31 00:00:00,0.268895
50%,2024-03-11 00:00:00,2024-03-24 00:00:00,0.315266
75%,2024-08-01 00:00:00,2024-08-11 00:00:00,0.380208
max,2024-12-23 00:00:00,2024-12-31 00:00:00,0.940476
std,NaN,NaN,0.114556


## FSPD30

In [54]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_cic_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    s_cic_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 44eb0a4e-efa8-484c-929c-bffd58fef9ab successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [55]:
gini_results = calculate_periodic_gini(df_s_cic_scorefspd30, 's_cic_score', 'ln_fspd30_flag', 'FSPD30')
s_cic_scoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFSPD30.shape}")
s_cic_scoreFSPD30.columns.values

The shape of dataframe after copy is:	(96, 7)


array(['start_date', 'end_date', 's_cic_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [56]:
s_cic_scoreFSPD30.describe()

,start_date,end_date,s_cic_score_FSPD30_gini
count,96,96,96.000000
mean,2024-02-26 06:15:00,2024-03-07 16:00:00,0.320292
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.066869
25%,2023-10-14 06:00:00,2023-10-27 06:00:00,0.270472
50%,2024-02-28 00:00:00,2024-03-06 12:00:00,0.319791
75%,2024-07-09 18:00:00,2024-07-22 18:00:00,0.359617
max,2024-11-25 00:00:00,2024-12-01 00:00:00,0.680714
std,NaN,NaN,0.087323


In [57]:
s_cic_scoreFSPD30.tail()

,start_date,end_date,s_cic_score_FSPD30_gini,period,Model_Name,version,bad_rate
91,2024-11-01,2024-11-30,0.263308,Month,s_cic_score,1.1.0,FSPD30
92,2024-11-04,2024-11-10,0.342195,Week,s_cic_score,1.1.0,FSPD30
93,2024-11-11,2024-11-17,0.202355,Week,s_cic_score,1.1.0,FSPD30
94,2024-11-18,2024-11-24,0.273222,Week,s_cic_score,1.1.0,FSPD30
95,2024-11-25,2024-12-01,0.105903,Week,s_cic_score,1.1.0,FSPD30


## FSTPD30

In [58]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_cic_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    s_cic_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID dff7f0c4-f3b9-4731-9f95-27a158f526aa successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [59]:
gini_results = calculate_periodic_gini(df_s_cic_scorefstpd30, 's_cic_score', 'ln_fstpd30_flag', 'FSTPD30')
s_cic_scoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFSTPD30.shape}")
s_cic_scoreFSTPD30.columns.values

The shape of dataframe after copy is:	(90, 7)


array(['start_date', 'end_date', 's_cic_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [60]:
s_cic_scoreFSTPD30.describe()

,start_date,end_date,s_cic_score_FSTPD30_gini
count,90,90,90.000000
mean,2024-02-09 02:56:00,2024-02-19 13:36:00,0.306488
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.154422
25%,2023-10-03 18:00:00,2023-10-16 18:00:00,0.257354
50%,2024-02-08 12:00:00,2024-02-21 12:00:00,0.300352
75%,2024-06-15 06:00:00,2024-06-28 06:00:00,0.325598
max,2024-10-21 00:00:00,2024-10-31 00:00:00,0.571813
std,NaN,NaN,0.073241


In [61]:
s_cic_scoreFSTPD30.tail()

,start_date,end_date,s_cic_score_FSTPD30_gini,period,Model_Name,version,bad_rate
85,2024-09-30,2024-10-06,0.313062,Week,s_cic_score,1.1.0,FSTPD30
86,2024-10-01,2024-10-31,0.302723,Month,s_cic_score,1.1.0,FSTPD30
87,2024-10-07,2024-10-13,0.321294,Week,s_cic_score,1.1.0,FSTPD30
88,2024-10-14,2024-10-20,0.271744,Week,s_cic_score,1.1.0,FSTPD30
89,2024-10-21,2024-10-27,0.299608,Week,s_cic_score,1.1.0,FSTPD30


# Combining s_cic_score

In [62]:
import functools

dataframes = [s_cic_scoreFPD10, s_cic_scoreFPD30, s_cic_scoreFSPD30, s_cic_scoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 's_cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 's_cic_score_FPD30_gini',
       's_cic_score_FSPD30_gini', 's_cic_score_FSTPD30_gini'],
      dtype=object)

In [63]:
final_df = final_df[['start_date', 'end_date', 'period',
       'Model_Name', 'version', 'bad_rate','s_cic_score_FPD10_gini','s_cic_score_FPD30_gini', 's_cic_score_FSPD30_gini', 's_cic_score_FSTPD30_gini']].copy()

## Creating the table

In [64]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_cic_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b0e794ed-a461-46d6-9581-c4aa91098e37>

In [65]:


import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('s_cic_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('s_cic_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('s_cic_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_cic_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_cic_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=10ecdb23-7e63-428b-99c5-15bcab52fd29>

# sb_stack_score

## FPD10

In [66]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_stack_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    sb_stack_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefpd10.head()

Job ID 86fa4be3-da08-4a4a-93ae-fd0d9550b0b0 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2023-06-01 10:42:34,f91cba31-7630-477a-8cdd-d010b63c4900,0.020090,0,1
1,2023-06-01 11:02:18,7e7e7678-b36d-4b26-ad01-272565b54780,0.055517,0,1
2,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.043838,0,1
3,2023-06-01 14:43:46,4e78ebf3-256e-426c-8840-a7f9e1403a97,0.084972,0,1
4,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.085976,0,1


In [67]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefpd10, 'sb_stack_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(106, 7)


array(['start_date', 'end_date', 'sb_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [68]:
M1FPD10.describe()

,start_date,end_date,sb_stack_score_FPD10_gini
count,106,106,106.000000
mean,2024-03-20 14:56:36.226415104,2024-03-31 01:35:05.660377344,0.348532
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.072727
25%,2023-10-24 18:00:00,2023-11-01 06:00:00,0.299407
50%,2024-03-21 12:00:00,2024-03-31 00:00:00,0.336644
75%,2024-08-17 06:00:00,2024-08-29 12:00:00,0.407867
max,2025-01-13 00:00:00,2025-01-31 00:00:00,0.663522
std,NaN,NaN,0.097246


In [69]:
M1FPD10.head()

,start_date,end_date,sb_stack_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.487907,Week,sb_stack_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.317960,Month,sb_stack_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.412426,Week,sb_stack_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.663522,Week,sb_stack_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,-0.072727,Week,sb_stack_score,1.1.0,FPD10


## FPD30

In [70]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_stack_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    sb_stack_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefpd30.head()

Job ID ac2f0fa5-b6d1-4461-8154-c46dbbeb382f successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.133378,0,1
1,2023-06-01 12:17:20,4fddb4fa-6e60-42b0-adec-295658bd1093,0.048930,0,1
2,2023-06-01 12:36:51,855bccd7-f3d4-461f-9748-bb50b07e0327,0.143019,0,1
3,2023-06-01 14:57:43,ac518432-9362-4014-86d4-565c0125b27b,0.078666,0,1
4,2023-06-01 15:52:56,b11e0b23-1b34-4d8c-984d-1239a7b389c1,0.023960,0,1


In [71]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefpd30, 'sb_stack_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(102, 7)


array(['start_date', 'end_date', 'sb_stack_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [72]:
M2FPD30.describe()

,start_date,end_date,sb_stack_score_FPD30_gini
count,102,102,102.000000
mean,2024-03-09 05:52:56.470588160,2024-03-19 15:03:31.764705792,0.369012
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.018742
25%,2023-10-17 18:00:00,2023-10-29 12:00:00,0.324044
50%,2024-03-07 12:00:00,2024-03-20 12:00:00,0.360321
75%,2024-07-31 06:00:00,2024-08-09 06:00:00,0.426560
max,2024-12-23 00:00:00,2024-12-31 00:00:00,0.639778
std,NaN,NaN,0.096305


In [73]:
M2FPD30.tail()

,start_date,end_date,sb_stack_score_FPD30_gini,period,Model_Name,version,bad_rate
97,2024-12-01,2024-12-31,0.330326,Month,sb_stack_score,1.1.0,FPD30
98,2024-12-02,2024-12-08,0.372214,Week,sb_stack_score,1.1.0,FPD30
99,2024-12-09,2024-12-15,0.305134,Week,sb_stack_score,1.1.0,FPD30
100,2024-12-16,2024-12-22,0.324492,Week,sb_stack_score,1.1.0,FPD30
101,2024-12-23,2024-12-29,0.296113,Week,sb_stack_score,1.1.0,FPD30


## FSPD30

In [74]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_stack_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    sb_stack_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefspd30.head()

Job ID a296c595-3869-42fa-a53a-97b005757093 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fspd30_flag,ln_mature_fspd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.133378,0,1
1,2023-06-01 12:17:20,4fddb4fa-6e60-42b0-adec-295658bd1093,0.048930,0,1
2,2023-06-01 12:36:51,855bccd7-f3d4-461f-9748-bb50b07e0327,0.143019,0,1
3,2023-06-01 14:57:43,ac518432-9362-4014-86d4-565c0125b27b,0.078666,0,1
4,2023-06-01 15:52:56,b11e0b23-1b34-4d8c-984d-1239a7b389c1,0.023960,0,1


In [75]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefspd30, 'sb_stack_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
M3FSPD30.columns.values

The shape of dataframe after copy is:	(97, 7)


array(['start_date', 'end_date', 'sb_stack_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [76]:
M3FSPD30.describe()

,start_date,end_date,sb_stack_score_FSPD30_gini
count,97,97,97.000000
mean,2024-02-24 00:29:41.443299072,2024-03-05 09:09:16.701030912,0.371971
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.128880
25%,2023-10-09 00:00:00,2023-10-22 00:00:00,0.325485
50%,2024-02-26 00:00:00,2024-03-03 00:00:00,0.351603
75%,2024-07-08 00:00:00,2024-07-21 00:00:00,0.422221
max,2024-11-25 00:00:00,2024-12-01 00:00:00,0.553537
std,NaN,NaN,0.076892


In [77]:
M3FSPD30.tail()

,start_date,end_date,sb_stack_score_FSPD30_gini,period,Model_Name,version,bad_rate
92,2024-11-01,2024-11-30,0.332077,Month,sb_stack_score,1.1.0,FSPD30
93,2024-11-04,2024-11-10,0.309879,Week,sb_stack_score,1.1.0,FSPD30
94,2024-11-11,2024-11-17,0.344365,Week,sb_stack_score,1.1.0,FSPD30
95,2024-11-18,2024-11-24,0.308463,Week,sb_stack_score,1.1.0,FSPD30
96,2024-11-25,2024-12-01,0.344112,Week,sb_stack_score,1.1.0,FSPD30


## FSTPD30

In [78]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_stack_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    sb_stack_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefstpd30.head()

Job ID 8f87de5d-eedf-47ec-8a95-9f9c077e8e2f successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
0,2023-06-01 10:42:34,f91cba31-7630-477a-8cdd-d010b63c4900,0.020090,0,1
1,2023-06-01 11:02:18,7e7e7678-b36d-4b26-ad01-272565b54780,0.055517,0,1
2,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.043838,0,1
3,2023-06-01 14:43:46,4e78ebf3-256e-426c-8840-a7f9e1403a97,0.084972,0,1
4,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.085976,0,1


In [79]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefstpd30, 'sb_stack_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
M4FSTPD30.columns.values

The shape of dataframe after copy is:	(91, 7)


array(['start_date', 'end_date', 'sb_stack_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [80]:
M4FSTPD30.describe()

,start_date,end_date,sb_stack_score_FSTPD30_gini
count,91,91,91.000000
mean,2024-02-06 22:09:13.846153728,2024-02-17 07:38:54.065934080,0.346911
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.149705
25%,2023-10-01 12:00:00,2023-10-11 12:00:00,0.308426
50%,2024-02-05 00:00:00,2024-02-18 00:00:00,0.338667
75%,2024-06-13 12:00:00,2024-06-26 12:00:00,0.389003
max,2024-10-21 00:00:00,2024-10-31 00:00:00,0.501592
std,NaN,NaN,0.061456


In [81]:
M4FSTPD30.tail()

,start_date,end_date,sb_stack_score_FSTPD30_gini,period,Model_Name,version,bad_rate
86,2024-09-30,2024-10-06,0.317665,Week,sb_stack_score,1.1.0,FSTPD30
87,2024-10-01,2024-10-31,0.308261,Month,sb_stack_score,1.1.0,FSTPD30
88,2024-10-07,2024-10-13,0.329735,Week,sb_stack_score,1.1.0,FSTPD30
89,2024-10-14,2024-10-20,0.292830,Week,sb_stack_score,1.1.0,FSTPD30
90,2024-10-21,2024-10-27,0.314181,Week,sb_stack_score,1.1.0,FSTPD30


## Combining the dataframes

In [82]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sb_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'sb_stack_score_FPD30_gini',
       'sb_stack_score_FSPD30_gini', 'sb_stack_score_FSTPD30_gini'],
      dtype=object)

In [83]:
final_df = final_df[['start_date', 'end_date', 'period', 'Model_Name', 'version', 'bad_rate', 'sb_stack_score_FPD10_gini', 'sb_stack_score_FPD30_gini',  'sb_stack_score_FSPD30_gini', 'sb_stack_score_FSTPD30_gini']].copy()

## Creating the table 

In [84]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_stack_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=fcb074eb-3b26-4bce-ad58-5492480f4897>

In [85]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('sb_stack_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('sb_stack_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('sb_stack_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('sb_stack_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_stack_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b5b9f81f-71f3-45e6-9f40-822a19ef13d1>

# sa_stack_score

## FPD10

In [86]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sa_stack_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    sa_stack_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefpd10.head()

Job ID 68a003bf-12a5-4479-bb7b-e47e478a6e48 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.156864,0,1
1,2023-06-01 12:17:20,4fddb4fa-6e60-42b0-adec-295658bd1093,0.055236,0,1
2,2023-06-01 12:36:51,855bccd7-f3d4-461f-9748-bb50b07e0327,0.148419,0,1
3,2023-06-01 14:57:43,ac518432-9362-4014-86d4-565c0125b27b,0.114224,0,1
4,2023-06-01 15:52:56,b11e0b23-1b34-4d8c-984d-1239a7b389c1,0.031071,0,1


In [87]:
sa_stack_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.156864,0,1
1,2023-06-01 12:17:20,4fddb4fa-6e60-42b0-adec-295658bd1093,0.055236,0,1
2,2023-06-01 12:36:51,855bccd7-f3d4-461f-9748-bb50b07e0327,0.148419,0,1
3,2023-06-01 14:57:43,ac518432-9362-4014-86d4-565c0125b27b,0.114224,0,1
4,2023-06-01 15:52:56,b11e0b23-1b34-4d8c-984d-1239a7b389c1,0.031071,0,1


In [88]:
gini_results = calculate_periodic_gini(sa_stack_scorefpd10, 'sa_stack_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(106, 7)


array(['start_date', 'end_date', 'sa_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [89]:
M1FPD10.describe()

,start_date,end_date,sa_stack_score_FPD10_gini
count,106,106,106.000000
mean,2024-03-20 14:56:36.226415104,2024-03-31 01:35:05.660377344,0.370577
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.063030
25%,2023-10-24 18:00:00,2023-11-01 06:00:00,0.328960
50%,2024-03-21 12:00:00,2024-03-31 00:00:00,0.357001
75%,2024-08-17 06:00:00,2024-08-29 12:00:00,0.429712
max,2025-01-13 00:00:00,2025-01-31 00:00:00,0.663522
std,NaN,NaN,0.098684


In [90]:
M1FPD10.head()

,start_date,end_date,sa_stack_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.549625,Week,sa_stack_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.344488,Month,sa_stack_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.466623,Week,sa_stack_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.663522,Week,sa_stack_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,-0.063030,Week,sa_stack_score,1.1.0,FPD10


## FPD30

In [91]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sa_stack_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    sa_stack_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefpd30.head()

Job ID b35c46fa-a957-45bf-9797-abde48d6d272 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-06-01 10:42:34,f91cba31-7630-477a-8cdd-d010b63c4900,0.027921,0,1
1,2023-06-01 11:02:18,7e7e7678-b36d-4b26-ad01-272565b54780,0.046504,0,1
2,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.049986,0,1
3,2023-06-01 14:43:46,4e78ebf3-256e-426c-8840-a7f9e1403a97,0.094184,0,1
4,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.089339,0,1


In [92]:
sa_stack_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-06-01 10:42:34,f91cba31-7630-477a-8cdd-d010b63c4900,0.027921,0,1
1,2023-06-01 11:02:18,7e7e7678-b36d-4b26-ad01-272565b54780,0.046504,0,1
2,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.049986,0,1
3,2023-06-01 14:43:46,4e78ebf3-256e-426c-8840-a7f9e1403a97,0.094184,0,1
4,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.089339,0,1


In [93]:
gini_results = calculate_periodic_gini(sa_stack_scorefpd30, 'sa_stack_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(102, 7)


array(['start_date', 'end_date', 'sa_stack_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [94]:
M2FPD30.describe()

,start_date,end_date,sa_stack_score_FPD30_gini
count,102,102,102.000000
mean,2024-03-09 05:52:56.470588160,2024-03-19 15:03:31.764705792,0.393343
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.033467
25%,2023-10-17 18:00:00,2023-10-29 12:00:00,0.354627
50%,2024-03-07 12:00:00,2024-03-20 12:00:00,0.387955
75%,2024-07-31 06:00:00,2024-08-09 06:00:00,0.449207
max,2024-12-23 00:00:00,2024-12-31 00:00:00,0.634656
std,NaN,NaN,0.097187


In [95]:
M2FPD30.head()

,start_date,end_date,sa_stack_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.512727,Week,sa_stack_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,0.331562,Month,sa_stack_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.417778,Week,sa_stack_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,0.634656,Week,sa_stack_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,-0.033467,Week,sa_stack_score,1.1.0,FPD30


## FSTPD30

In [96]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sa_stack_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    sa_stack_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefspd30.head()

Job ID 42bea033-7d30-4b0d-891e-171574a70cca successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fspd30_flag,ln_mature_fspd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.156864,0,1
1,2023-06-01 12:17:20,4fddb4fa-6e60-42b0-adec-295658bd1093,0.055236,0,1
2,2023-06-01 12:36:51,855bccd7-f3d4-461f-9748-bb50b07e0327,0.148419,0,1
3,2023-06-01 14:57:43,ac518432-9362-4014-86d4-565c0125b27b,0.114224,0,1
4,2023-06-01 15:52:56,b11e0b23-1b34-4d8c-984d-1239a7b389c1,0.031071,0,1


In [97]:
gini_results = calculate_periodic_gini(sa_stack_scorefspd30, 'sa_stack_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
M3FSPD30.columns.values

The shape of dataframe after copy is:	(97, 7)


array(['start_date', 'end_date', 'sa_stack_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [98]:
M3FSPD30.tail()

,start_date,end_date,sa_stack_score_FSPD30_gini,period,Model_Name,version,bad_rate
92,2024-11-01,2024-11-30,0.355513,Month,sa_stack_score,1.1.0,FSPD30
93,2024-11-04,2024-11-10,0.348228,Week,sa_stack_score,1.1.0,FSPD30
94,2024-11-11,2024-11-17,0.357016,Week,sa_stack_score,1.1.0,FSPD30
95,2024-11-18,2024-11-24,0.337728,Week,sa_stack_score,1.1.0,FSPD30
96,2024-11-25,2024-12-01,0.324588,Week,sa_stack_score,1.1.0,FSPD30


## FSTPD30

In [99]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sa_stack_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    sa_stack_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefstpd30.head()

Job ID 80f19c8a-326a-4d23-a139-decb64b26b78 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.156864,0,1
1,2023-06-01 12:17:20,4fddb4fa-6e60-42b0-adec-295658bd1093,0.055236,0,1
2,2023-06-01 12:36:51,855bccd7-f3d4-461f-9748-bb50b07e0327,0.148419,0,1
3,2023-06-01 14:57:43,ac518432-9362-4014-86d4-565c0125b27b,0.114224,0,1
4,2023-06-01 15:52:56,b11e0b23-1b34-4d8c-984d-1239a7b389c1,0.031071,0,1


In [100]:
sa_stack_scorefstpd30.tail()

,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
124268,2024-10-25 18:07:28,cef44691-c5cb-48c8-8715-b84703fcceae,0.040753,0,1
124269,2024-10-25 18:17:03,a8968b6a-fc0a-41bb-b360-76400145242d,0.243156,0,1
124270,2024-10-25 18:55:40,00382444-7f98-4514-9dba-8c7a75438225,0.044918,0,1
124271,2024-10-25 19:03:28,69c78eae-2c19-4718-bd65-9238df2f2ebc,0.055356,0,1
124272,2024-10-25 20:35:25,35785f03-b84f-4b25-9fa9-d9b29bcab2ea,0.074039,0,1


In [101]:
gini_results = calculate_periodic_gini(sa_stack_scorefstpd30, 'sa_stack_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
M4FSTPD30.columns.values

The shape of dataframe after copy is:	(91, 7)


array(['start_date', 'end_date', 'sa_stack_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

## Combining the dataframes

In [102]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sa_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'sa_stack_score_FPD30_gini',
       'sa_stack_score_FSPD30_gini', 'sa_stack_score_FSTPD30_gini'],
      dtype=object)

In [103]:
final_df = final_df[['start_date', 'end_date', 'period', 'Model_Name', 'version', 'bad_rate', 'sa_stack_score_FPD10_gini', 'sa_stack_score_FPD30_gini',  'sa_stack_score_FSPD30_gini', 'sa_stack_score_FSTPD30_gini']].copy()

## Creating the table 

In [104]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sa_stack_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a8af38c7-7e79-416e-b309-cb2f4c36a3da>

In [105]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('sa_stack_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('sa_stack_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('sa_stack_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('sa_stack_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sa_stack_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ba2dffdd-85bf-4d8a-aece-85a6fd21c790>

# gen_credo_score

## FPD10

In [106]:


sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    gen_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    gen_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

gen_credo_scorefpd10.head()

Job ID 620c05eb-47db-4fe3-acb0-76426183f217 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,gen_credo_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2023-01-02 10:56:20,ac41223e-0d42-4569-b21f-789bce021291,0.072991,0,1
1,2023-01-02 11:39:36,d438dd4f-4c6c-44b5-ba83-58585ba8530a,0.150079,0,1
2,2023-01-02 12:04:44,44247f51-ac7a-4035-8d34-af7af2d9548c,0.099274,1,1
3,2023-01-02 12:13:00,5e85ce9b-fb16-48a4-9f62-c543782b54e1,0.152459,0,1
4,2023-01-02 13:46:09,cfb70c1e-bd39-4202-a376-fc003a6b231d,0.047018,0,1


In [107]:
gini_results = calculate_periodic_gini(gen_credo_scorefpd10, 'gen_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(132, 7)


array(['start_date', 'end_date', 'gen_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [108]:
M1FPD10.head()

,start_date,end_date,gen_credo_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.246725,Month,gen_credo_score,1.1.0,FPD10
1,2023-01-02,2023-01-08,0.301125,Week,gen_credo_score,1.1.0,FPD10
2,2023-01-09,2023-01-15,0.273400,Week,gen_credo_score,1.1.0,FPD10
3,2023-01-16,2023-01-22,0.667532,Week,gen_credo_score,1.1.0,FPD10
4,2023-01-23,2023-01-29,0.086505,Week,gen_credo_score,1.1.0,FPD10


## FPD30

In [109]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    gen_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    gen_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

gen_credo_scorefpd30.head()

Job ID 42adbe4c-aa55-4ca3-93e6-7be61f72ca1e successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,gen_credo_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-01-02 10:19:54,73902306-ce07-4625-9d1e-a611eeb83166,0.175299,0,1
1,2023-01-02 12:28:35,92a04afd-ec2e-4941-a1be-0e3af52aab1f,0.074451,0,1
2,2023-01-02 12:44:11,f0a9e5c7-560d-46a3-9c4c-16d518d1a605,0.187076,0,1
3,2023-01-02 12:53:05,6c53ccd6-5de3-4b2a-8216-caa1b1146cd6,0.100128,1,1
4,2023-01-02 13:19:37,ea076112-bc60-4a12-9463-37a9be7da3e2,0.169417,0,1


In [110]:
gen_credo_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,gen_credo_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-01-02 10:19:54,73902306-ce07-4625-9d1e-a611eeb83166,0.175299,0,1
1,2023-01-02 12:28:35,92a04afd-ec2e-4941-a1be-0e3af52aab1f,0.074451,0,1
2,2023-01-02 12:44:11,f0a9e5c7-560d-46a3-9c4c-16d518d1a605,0.187076,0,1
3,2023-01-02 12:53:05,6c53ccd6-5de3-4b2a-8216-caa1b1146cd6,0.100128,1,1
4,2023-01-02 13:19:37,ea076112-bc60-4a12-9463-37a9be7da3e2,0.169417,0,1


In [111]:
gini_results = calculate_periodic_gini(gen_credo_scorefpd30, 'gen_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(128, 7)


array(['start_date', 'end_date', 'gen_credo_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [112]:
M2FPD30.head()

,start_date,end_date,gen_credo_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.253005,Month,gen_credo_score,1.1.0,FPD30
1,2023-01-02,2023-01-08,0.342149,Week,gen_credo_score,1.1.0,FPD30
2,2023-01-09,2023-01-15,0.407163,Week,gen_credo_score,1.1.0,FPD30
3,2023-01-16,2023-01-22,0.667532,Week,gen_credo_score,1.1.0,FPD30
4,2023-01-23,2023-01-29,0.025547,Week,gen_credo_score,1.1.0,FPD30


## FSPD30

In [113]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    gen_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    gen_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(gen_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(gen_credo_scorefspd30, 'gen_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

Job ID e7ba860e-8034-4c6f-b630-f3fac9c44cc0 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  gen_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166         0.175299   
1 2023-01-02 12:28:35  92a04afd-ec2e-4941-a1be-0e3af52aab1f         0.074451   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'gen_credo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,gen_credo_score_FSPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.172055,Month,gen_credo_score,1.1.0,FSPD30
1,2023-01-02,2023-01-08,0.262803,Week,gen_credo_score,1.1.0,FSPD30
2,2023-01-09,2023-01-15,0.189744,Week,gen_credo_score,1.1.0,FSPD30
3,2023-01-16,2023-01-22,0.410738,Week,gen_credo_score,1.1.0,FSPD30
4,2023-01-23,2023-01-29,0.073579,Week,gen_credo_score,1.1.0,FSPD30


## FSTPD30

In [114]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    gen_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    gen_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(gen_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(gen_credo_scorefstpd30, 'gen_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()

Job ID d57e6b17-1b83-4db9-b21d-73bb2a7ed2f4 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  gen_credo_score  \
0 2023-01-02 10:56:20  ac41223e-0d42-4569-b21f-789bce021291         0.072991   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a         0.150079   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'gen_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,gen_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.180837,Month,gen_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.282959,Week,gen_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.189908,Week,gen_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.198840,Week,gen_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.162845,Week,gen_credo_score,1.1.0,FSTPD30


## Combining tables 

In [115]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'gen_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'gen_credo_score_FPD30_gini',
       'gen_credo_score_FSPD30_gini', 'gen_credo_score_FSTPD30_gini'],
      dtype=object)

In [116]:
final_df = final_df[['start_date', 'end_date','period',
       'Model_Name', 'version', 'bad_rate', 'gen_credo_score_FPD10_gini','gen_credo_score_FPD30_gini',   'gen_credo_score_FSPD30_gini', 'gen_credo_score_FSTPD30_gini']].copy()

In [117]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_gen_credo_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d7b46b58-20c0-4519-a0e3-8f8392634e31>

In [118]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('gen_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('gen_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('gen_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('gen_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_gen_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a5f466d7-7b68-4204-8b73-baf77cf615a9>

# c_credo_score

In [119]:
sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    c_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    c_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

c_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(c_credo_scorefpd10, 'c_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    c_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    c_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

c_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(c_credo_scorefpd30, 'c_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    c_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    c_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(c_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(c_credo_scorefspd30, 'c_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    c_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    c_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(c_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(c_credo_scorefstpd30, 'c_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 58efdb75-45be-4683-bfc0-35a69e4ac5a7 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID 661086f5-4f5f-4f90-9226-8d7973c01074 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID 9fbc5b61-664f-4404-baf1-8267cf965bd7 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  c_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166       0.198732   
1 2023-01-02 12:28:35  92a04afd-ec2e-4941-a1be-0e3af52aab1f       0.281711   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'c_credo_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID 466c48c5-7f75-4c89-b7dc-e8e2aedf7528 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  c_credo_score  \
0 2023-01-02 10:56:20  ac41223e-0d42-4569-b21f-789bce021291        0.20584   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a        0.19817   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'c_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,c_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.035948,Month,c_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.036009,Week,c_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.062143,Week,c_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.275891,Week,c_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,-0.050558,Week,c_credo_score,1.1.0,FSTPD30


In [120]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'c_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'c_credo_score_FPD30_gini',
       'c_credo_score_FSPD30_gini', 'c_credo_score_FSTPD30_gini'],
      dtype=object)

In [121]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate','c_credo_score_FPD10_gini', 'c_credo_score_FPD30_gini',    'c_credo_score_FSPD30_gini', 'c_credo_score_FSTPD30_gini']].copy()

In [122]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_c_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8aa9198c-0945-4947-b206-e151d46beafc>

In [123]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('c_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('c_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('c_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('c_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_c_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=dfacf457-9c56-4249-85e8-49be1c985564>

# s_credo_score

In [124]:


sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    s_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

s_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(s_credo_scorefpd10, 's_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    s_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

s_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(s_credo_scorefpd30, 's_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    s_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(s_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(s_credo_scorefspd30, 's_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    s_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(s_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(s_credo_scorefstpd30, 's_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID a850d2e5-1218-437b-bda1-ffc1f72083af successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID 176eae4b-c8e7-4a82-96b3-d086a0314e10 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID 6a9ce448-9866-4810-a55d-20524b27ec5d successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  s_credo_score  \
0 2023-01-02 10:56:20  ac41223e-0d42-4569-b21f-789bce021291       0.093295   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a       0.122682   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 's_credo_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID 935651b4-83a1-48e9-a06f-ea19cd4fc6ea successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  s_credo_score  \
0 2023-01-02 10:56:20  ac41223e-0d42-4569-b21f-789bce021291       0.093295   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a       0.122682   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 's_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,s_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.219086,Month,s_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.261708,Week,s_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.251056,Week,s_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.275062,Week,s_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.157554,Week,s_credo_score,1.1.0,FSTPD30


## Combining data

In [125]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 's_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 's_credo_score_FPD30_gini',
       's_credo_score_FSPD30_gini', 's_credo_score_FSTPD30_gini'],
      dtype=object)

In [126]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate','s_credo_score_FPD10_gini', 's_credo_score_FPD30_gini',    's_credo_score_FSPD30_gini', 's_credo_score_FSTPD30_gini']].copy()

In [127]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_credo_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=5aaad934-835f-43dc-b889-ace3dac93bef>

In [128]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('s_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('s_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('s_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b6962150-c310-4e67-b7be-780413f5fbca>

In [129]:
final_df.head()

,start_date,end_date,period,Model_Name,version,bad_rate,s_credo_score_FPD10_gini,s_credo_score_FPD30_gini,s_credo_score_FSPD30_gini,s_credo_score_FSTPD30_gini
0,2023-01-01,2023-01-31,Month,s_credo_score,1.1.0,FPD10,0.230181,NaN,NaN,NaN
1,2023-01-02,2023-01-08,Week,s_credo_score,1.1.0,FPD10,0.248978,NaN,NaN,NaN
2,2023-01-09,2023-01-15,Week,s_credo_score,1.1.0,FPD10,0.296337,NaN,NaN,NaN
3,2023-01-16,2023-01-22,Week,s_credo_score,1.1.0,FPD10,0.529870,NaN,NaN,NaN
4,2023-01-23,2023-01-29,Week,s_credo_score,1.1.0,FPD10,0.079585,NaN,NaN,NaN


# fu_credo_score

In [130]:


sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    fu_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    fu_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

fu_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(fu_credo_scorefpd10, 'fu_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    fu_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    fu_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

fu_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(fu_credo_scorefpd30, 'fu_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    fu_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    fu_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(fu_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(fu_credo_scorefspd30, 'fu_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    fu_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    fu_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(fu_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(fu_credo_scorefstpd30, 'fu_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 40c4da86-c40c-4b1e-9af6-602b6ce1b0ca successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID 32d32bcb-7bb1-4026-8258-c21d0f752fbb successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID da76cdf2-fbb4-40b7-ad71-e32722922b3d successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  fu_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166        0.096835   
1 2023-01-02 12:28:35  92a04afd-ec2e-4941-a1be-0e3af52aab1f        0.054208   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'fu_credo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID a83063f0-92df-400f-a8c8-aaca46438169 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  fu_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166        0.096835   
1 2023-01-02 12:28:35  92a04afd-ec2e-4941-a1be-0e3af52aab1f        0.054208   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'fu_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,fu_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.072860,Month,fu_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.002165,Week,fu_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.056674,Week,fu_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.326429,Week,fu_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.109935,Week,fu_credo_score,1.1.0,FSTPD30


## Combining data

In [131]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'fu_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'fu_credo_score_FPD30_gini',
       'fu_credo_score_FSPD30_gini', 'fu_credo_score_FSTPD30_gini'],
      dtype=object)

In [132]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate','fu_credo_score_FPD10_gini', 'fu_credo_score_FPD30_gini',    'fu_credo_score_FSPD30_gini', 'fu_credo_score_FSTPD30_gini']].copy()

In [133]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_fu_credo_score"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=20bf236a-67f4-4991-8a63-065b40869100>

In [134]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('fu_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('fu_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('fu_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('fu_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_fu_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a2a2ae3e-d4ce-47b1-a142-5249dd5421c6>

In [135]:
final_df.head()

,start_date,end_date,period,Model_Name,version,bad_rate,fu_credo_score_FPD10_gini,fu_credo_score_FPD30_gini,fu_credo_score_FSPD30_gini,fu_credo_score_FSTPD30_gini
0,2023-01-01,2023-01-31,Month,fu_credo_score,1.1.0,FPD10,0.023608,NaN,NaN,NaN
1,2023-01-02,2023-01-08,Week,fu_credo_score,1.1.0,FPD10,-0.032464,NaN,NaN,NaN
2,2023-01-09,2023-01-15,Week,fu_credo_score,1.1.0,FPD10,0.016278,NaN,NaN,NaN
3,2023-01-16,2023-01-22,Week,fu_credo_score,1.1.0,FPD10,0.153247,NaN,NaN,NaN
4,2023-01-23,2023-01-29,Week,fu_credo_score,1.1.0,FPD10,0.120242,NaN,NaN,NaN


# r_credo_score

In [136]:


sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    r_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    r_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

r_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(r_credo_scorefpd10, 'r_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    r_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    r_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

r_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(r_credo_scorefpd30, 'r_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    r_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    r_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(r_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(r_credo_scorefspd30, 'r_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    r_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    r_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(r_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(r_credo_scorefstpd30, 'r_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 6b0fcbce-c385-48fc-86d5-577fd00ce285 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID 5984e31d-cb56-4adc-b9b5-4cd98ee91985 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID 3405559a-2ba5-4ccc-9401-f2c7779aa1ba successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  r_credo_score  \
0 2023-01-02 10:56:20  ac41223e-0d42-4569-b21f-789bce021291       0.328053   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a       0.150079   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'r_credo_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID eeaa3b46-f05b-4cc9-9b27-a8d3ec475126 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  r_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166       0.276277   
1 2023-01-02 12:28:35  92a04afd-ec2e-4941-a1be-0e3af52aab1f       0.112691   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'r_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,r_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.048984,Month,r_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.014955,Week,r_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.193885,Week,r_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,-0.019056,Week,r_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.024691,Week,r_credo_score,1.1.0,FSTPD30


## Combining data

In [137]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'r_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'r_credo_score_FPD30_gini',
       'r_credo_score_FSPD30_gini', 'r_credo_score_FSTPD30_gini'],
      dtype=object)

In [138]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','r_credo_score_FPD10_gini','r_credo_score_FPD30_gini',    'r_credo_score_FSPD30_gini', 'r_credo_score_FSTPD30_gini']].copy()

In [139]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_r_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0efb9f3a-67e0-42f3-b4b2-507c1172f60b>

In [140]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('r_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('r_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('r_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('r_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_r_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=aa912535-0e11-49b3-a586-a94e30c9f374>

# old_gen_credo_score

In [141]:


sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_gen_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    old_gen_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_gen_credo_scorefpd10.head()

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefpd10, 'old_gen_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_gen_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    old_gen_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_gen_credo_scorefpd30.head()

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefpd30, 'old_gen_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_gen_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    old_gen_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_gen_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefspd30, 'old_gen_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_gen_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    old_gen_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_gen_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefstpd30, 'old_gen_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 12159e5a-d843-4a35-a522-19f44be1ba1b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID 511c5b06-22f4-42c2-8729-1c5abbc3021c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID e43c975d-cb61-44ba-b20a-6db141d5df1c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2023-01-02 12:44:11  f0a9e5c7-560d-46a3-9c4c-16d518d1a605   
1 2023-01-02 12:53:05  6c53ccd6-5de3-4b2a-8216-caa1b1146cd6   

  old_gen_credo_score  ln_fspd30_flag  ln_mature_fspd30_flag  
0               510.0               0                      1  
1               481.0               1                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'old_gen_credo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID c32515c9-0fe0-47ba-bda8-1a496ff360b0 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2023-01-02 12:44:11  f0a9e5c7-560d-46a3-9c4c-16d518d1a605   
1 2023-01-02 12:53:05  6c53ccd6-5de3-4b2a-8216-caa1b1146cd6   

  old_gen_credo_score  ln_fstpd30_flag  ln_mature_fstpd30_flag  
0               510.0                0                       1  
1               481.0                1                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'old_gen_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,old_gen_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.122077,Month,old_gen_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.230331,Week,old_gen_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.148605,Week,old_gen_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.130184,Week,old_gen_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.000744,Week,old_gen_credo_score,1.1.0,FSTPD30


## Combining data

In [142]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'old_gen_credo_score_FPD10_gini',
       'period', 'Model_Name', 'version', 'bad_rate',
       'old_gen_credo_score_FPD30_gini',
       'old_gen_credo_score_FSPD30_gini',
       'old_gen_credo_score_FSTPD30_gini'], dtype=object)

In [143]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','old_gen_credo_score_FPD10_gini','old_gen_credo_score_FPD30_gini',    'old_gen_credo_score_FSPD30_gini', 'old_gen_credo_score_FSTPD30_gini']].copy()

In [144]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8997bc39-abee-4103-be8d-dfbedeccc479>

In [145]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('old_gen_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('old_gen_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('old_gen_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('old_gen_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d5ce51b2-2895-4c2e-b34e-96f3e73f5d7b>

In [146]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 3987c30d-e3f2-48f5-a655-a67ad9e78567 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [147]:
df['bad_rate'].value_counts()

bad_rate
FPD10      132
FPD30      128
FSPD30     123
FSTPD30    117
Name: count, dtype: int64

# old_cic_score

In [148]:


sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_cic_scorefpd10.head()

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefpd10, 'old_cic_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_cic_scorefpd30.head()

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefpd30, 'old_cic_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_cic_scorefspd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefspd30, 'old_cic_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_cic_scorefstpd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefstpd30, 'old_cic_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 4dfb9802-f38b-4bb3-986b-6d1b408acbad successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(92, 7)
Job ID 49f370f2-b449-47f7-8c86-602a1af86d69 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(88, 7)
Job ID feff76c4-cbde-4558-8c05-6fc70f2bf98c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId old_cic_score  \
0 2023-08-23 10:30:20  799382a3-b224-48b5-ad0b-ac075c638651       484.000   
1 2023-08-23 15:36:50  c8e6977b-a5ca-48b0-9ef0-e6e9581a7229       484.000   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               1                      1  
1               0                      1  
The shape of dataframe after copy is:	(83, 7)
['start_date' 'end_date' 'old_cic_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID 97d2d910-bbaf-490a-be65-427237b130d8 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId old_cic_score  \
0 2023-08-23 20:04:02  9c7f4086-33c8-4b8f-b1b1-f95cc0b104b2       480.000   
1 2023-08-23 20:32:30  93f8687b-b8fe-4fcd-a5ee-58c29003183b       484.000   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                1                       1  
1                0                       1  
The shape of dataframe after copy is:	(77, 7)
['start_date' 'end_date' 'old_cic_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,old_cic_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-08-01,2023-08-31,0.104194,Month,old_cic_score,1.1.0,FSTPD30
1,2023-08-21,2023-08-27,0.086897,Week,old_cic_score,1.1.0,FSTPD30
2,2023-08-28,2023-09-03,0.175026,Week,old_cic_score,1.1.0,FSTPD30
3,2023-09-01,2023-09-30,0.196606,Month,old_cic_score,1.1.0,FSTPD30
4,2023-09-04,2023-09-10,0.171044,Week,old_cic_score,1.1.0,FSTPD30


## Combining data

In [149]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'old_cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'old_cic_score_FPD30_gini',
       'old_cic_score_FSPD30_gini', 'old_cic_score_FSTPD30_gini'],
      dtype=object)

In [150]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','old_cic_score_FPD10_gini','old_cic_score_FPD30_gini',    'old_cic_score_FSPD30_gini', 'old_cic_score_FSTPD30_gini']].copy()

In [151]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_cic_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=7fc4d8e5-1a24-475e-b6f6-c9254959571d>

In [152]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('old_cic_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('old_cic_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('old_cic_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('old_cic_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_cic_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ce59050c-f23c-447b-929f-da6a6b2ddb49>

# old_demo_score

In [153]:


sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_demo_scorefpd10.head()

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefpd10, 'old_demo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_demo_scorefpd30.head()

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefpd30, 'old_demo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_demo_scorefspd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefspd30, 'old_demo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_demo_scorefstpd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefstpd30, 'old_demo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID ddc67f80-3698-40c6-9523-b0f72cda4daf successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID 12f1698e-ad2f-4d20-8ee2-7456e003f0aa successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID 6636aaa1-f3de-4658-86c9-84e09639f001 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  old_demo_score  \
0 2023-01-02 10:56:20  ac41223e-0d42-4569-b21f-789bce021291           402.0   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a           419.0   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'old_demo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 212043f0-214d-4e2a-a724-5444dfa32493 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  old_demo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166           402.0   
1 2023-01-02 12:28:35  92a04afd-ec2e-4941-a1be-0e3af52aab1f           411.0   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'old_demo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,old_demo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.126121,Month,old_demo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.174341,Week,old_demo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.041511,Week,old_demo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.134217,Week,old_demo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.128160,Week,old_demo_score,1.1.0,FSTPD30


## Combining data

In [154]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'old_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'old_demo_score_FPD30_gini',
       'old_demo_score_FSPD30_gini', 'old_demo_score_FSTPD30_gini'],
      dtype=object)

In [155]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','old_demo_score_FPD10_gini','old_demo_score_FPD30_gini',    'old_demo_score_FSPD30_gini', 'old_demo_score_FSTPD30_gini']].copy()

In [156]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_demo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3d0f7734-e98e-4c58-b2c8-307f66e2d0fe>

In [157]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('old_demo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('old_demo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('old_demo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('old_demo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_demo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=97eb4e6a-bdcd-4c45-934e-18ba5c176a97>

# bu_bureau_score

In [158]:
sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fpd10_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

bu_bureau_scorefpd10.head()

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefpd10, 'bu_bureau_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fpd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

bu_bureau_scorefpd30.head()

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefpd30, 'bu_bureau_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fspd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(bu_bureau_scorefspd30.head(2))

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefspd30, 'bu_bureau_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fstpd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(bu_bureau_scorefstpd30.head(2))

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefstpd30, 'bu_bureau_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 901a4725-5267-46ae-a949-d91a49de575d successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(92, 7)
Job ID 39950d1c-7f87-4449-b80c-366b007c7e5b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(92, 7)
Job ID 4bda9a6d-31dc-48a2-b7a9-dd4593415267 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  bu_bureau_score  \
0 2023-01-10 16:49:53  f7506ebf-ba0b-40a4-b0bd-8ec91ea0ab96            339.0   
1 2023-01-12 17:50:30  a2fd2c0e-ad17-48e2-aac9-b6a22ad444d8            357.0   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               1                      1  
1               0                      1  
The shape of dataframe after copy is:	(92, 7)
['start_date' 'end_date' 'bu_bureau_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 3646cb00-1a90-40dc-9b77-12155c7a0c81 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  bu_bureau_score  \
0 2023-01-10 13:04:21  a9a82293-c02a-44b9-a901-433ed8867b91            369.0   
1 2023-01-13 18:19:41  546fd78e-d4c0-4eb6-8667-00aa187b0ddd            401.0   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(92, 7)
['start_date' 'end_date' 'bu_bureau_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,bu_bureau_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.031250,Month,bu_bureau_score,1.1.0,FSTPD30
1,2023-01-09,2023-01-15,-0.053333,Week,bu_bureau_score,1.1.0,FSTPD30
2,2023-01-16,2023-01-22,-0.833333,Week,bu_bureau_score,1.1.0,FSTPD30
3,2023-01-23,2023-01-29,0.416667,Week,bu_bureau_score,1.1.0,FSTPD30
4,2023-01-30,2023-02-05,0.266667,Week,bu_bureau_score,1.1.0,FSTPD30


## Combining data

In [159]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'bu_bureau_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'bu_bureau_score_FPD30_gini',
       'bu_bureau_score_FSPD30_gini', 'bu_bureau_score_FSTPD30_gini'],
      dtype=object)

In [160]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','bu_bureau_score_FPD10_gini','bu_bureau_score_FPD30_gini',    'bu_bureau_score_FSPD30_gini', 'bu_bureau_score_FSTPD30_gini']].copy()

In [161]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_bu_bureau_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=898aa4b7-6282-48d9-bd98-156ac485eff4>

In [162]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('bu_bureau_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('bu_bureau_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('bu_bureau_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('bu_bureau_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_bu_bureau_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=952cbe39-4459-4b1c-8d7a-f4397b31e68b>